In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from pyspark.sql import SparkSession
from tqdm import tqdm
import time  # to simulate loading for tqdm
import sys
import os
import glob
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col, to_date, count, min, max, lit
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType
from pyspark.sql.functions import col, sum as spark_sum, when

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import src.utils.bronze
import src.utils.silver
import src.utils.gold


print('Done Importing!')

Done Importing!


In [2]:
# # Create gold table and store to Google Cloud Storage
# gold_label_file_path = "Gold Layer/labels.csv"
# gold_label_gcs_path = f"gs://{bucket_name}/{gold_label_file_path}"

# try:
#     df_labels.to_csv(gold_label_gcs_path, index=False)
#     print("labels.csv Stored to Gold Layer Successfully! ✅")
# except Exception as e:
#     print(f"labels.csv Store Failed: {e}")

# Setup Hadoop
Go to your working directory: CS611_MLE_GROUP_PROJECT <br/>
Then, run this command in your VM terminal:
> wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.20/gcs-connector-hadoop3-2.2.20-shaded.jar -P jars/

<br/><br/>
Copy your google cloud application_default_credentials.json to this working directory, then can run the following commands:

In [3]:

spark = SparkSession \
    .builder \
    .config("spark.jars", "jars/gcs-connector-hadoop3-2.2.20-shaded.jar") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


25/06/18 07:22:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
# This is required if you are using service account and set true, 
spark._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.enable', 'true')
spark._jsc.hadoopConfiguration().set('google.cloud.auth.service.account.json.keyfile', "application_default_credentials.json")
# Following are required if you are using oAuth
# spark._jsc.hadoopConfiguration().set('fs.gs.auth.client.id', 'YOUR_OAUTH_CLIENT_ID')
# spark._jsc.hadoopConfiguration().set('fs.gs.auth.client.secret', 'OAUTH_SECRET')

# Partition Existing Bronze

In [5]:
# Setup Config
snapshot_date_str = "2015-01-01"

start_date_str = "2015-01-01"
end_date_str = "2017-03-31"

In [6]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
print("Done generate list of dates!")

Done generate list of dates!


In [7]:
dates_str_lst

['2015-01-01',
 '2015-02-01',
 '2015-03-01',
 '2015-04-01',
 '2015-05-01',
 '2015-06-01',
 '2015-07-01',
 '2015-08-01',
 '2015-09-01',
 '2015-10-01',
 '2015-11-01',
 '2015-12-01',
 '2016-01-01',
 '2016-02-01',
 '2016-03-01',
 '2016-04-01',
 '2016-05-01',
 '2016-06-01',
 '2016-07-01',
 '2016-08-01',
 '2016-09-01',
 '2016-10-01',
 '2016-11-01',
 '2016-12-01',
 '2017-01-01',
 '2017-02-01',
 '2017-03-01']

In [8]:
bucket_name = "cs611_mle"

### Bronze Member

In [9]:
### ~~~ Bronze Member ~~~
# create bronze datalake
bronze_member_directory = "datamart/bronze/member"

In [10]:
# No need to be partitioned
utils.bronze.process_bronze_member(bucket_name, bronze_member_directory, spark)
print('Done Build Bronze Table - Member!')

row count: 50000


25/06/18 07:22:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, msno, city, bd, gender, registered_via, registration_init_time
 Schema: _c0, Unnamed: 0, msno, city, bd, gender, registered_via, registration_init_time
Expected: _c0 but found: 
CSV file: gs://cs611_mle/Bronze%20Layer/members_50k.csv


saved to: gs://cs611_mle/datamart/bronze/member/bronze_members.csv
Done Build Bronze Table - Member!


### Bronze User Log

In [11]:
### ~~~ User Log ~~~
# create bronze datalake
# create multiple csv files 
bronze_userlog_directory = "datamart/bronze/userlog"

In [12]:

# run bronze backfill
for date_str in dates_str_lst:
    utils.bronze.process_bronze_userlog(date_str, bucket_name, bronze_userlog_directory, spark)
    
print('Done Build Bronze Table - userlog!')

2015-01-01row count: 337141


25/06/18 07:23:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, msno, date, num_25, num_50, num_75, num_985, num_100, num_unq, total_secs
 Schema: _c0, Unnamed: 0, msno, date, num_25, num_50, num_75, num_985, num_100, num_unq, total_secs
Expected: _c0 but found: 
CSV file: gs://cs611_mle/Bronze%20Layer/user_logs_50k.csv
                                                                                

saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_01_01.csv


2015-02-01row count: 301479


25/06/18 07:23:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, msno, date, num_25, num_50, num_75, num_985, num_100, num_unq, total_secs
 Schema: _c0, Unnamed: 0, msno, date, num_25, num_50, num_75, num_985, num_100, num_unq, total_secs
Expected: _c0 but found: 
CSV file: gs://cs611_mle/Bronze%20Layer/user_logs_50k.csv
                                                                                

...

2017-02-01row count: 377934


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_02_01.csv


2017-03-01row count: 432095


25/06/18 07:37:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, msno, date, num_25, num_50, num_75, num_985, num_100, num_unq, total_secs
 Schema: _c0, Unnamed: 0, msno, date, num_25, num_50, num_75, num_985, num_100, num_unq, total_secs
Expected: _c0 but found: 
CSV file: gs://cs611_mle/Bronze%20Layer/user_logs_50k.csv
                                                                                

saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv
Done Build Bronze Table - userlog!


# Partition Existing Silver

This should have been done in Bronze

### Silver Transaction

In [13]:
### ~~~ Transaction ~~~
# create silver datalake
# create multiple csv files 
bronze_transaction_directory = "datamart/bronze/transaction"

In [14]:

# run bronze backfill
for date_str in dates_str_lst:
    utils.bronze.process_bronze_transaction_partition(date_str, bucket_name, bronze_transaction_directory, spark)
print('Done Partition Silver (Actually Bronze) Table - transaction!')

2015-01-01row count: 16177


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_01_01.csv


2015-02-01row count: 14710


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_02_01.csv


2015-03-01row count: 18392


...

2017-02-01row count: 19328


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_02_01.csv


2017-03-01row count: 1823


[Stage 329:=====================>                                   (3 + 5) / 8]

saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_03_01.csv
Done Partition Silver (Actually Bronze) Table - transaction!


# Process Feature Silver Table

### Member ~ Bronze to Silver Table

In [15]:
# create silver datalake
silver_member_directory = "datamart/silver/member/"

In [16]:
utils.silver.process_silver_table_member(bucket_name, bronze_member_directory, silver_member_directory, spark).toPandas().head()

loaded from: gs://cs611_mle/datamart/bronze/member/bronze_members.csv row count: 50000
+------+-----+
|gender|count|
+------+-----+
|  NULL|28551|
|  male|10973|
|female|10476|
+------+-----+



saved to: gs://cs611_mle/datamart/silver/member//silver_member.parquet


,msno,city,gender,registered_via,registration_init_time
0,/wRzURqhsFLj3mGJjHDE4g4VWpbw8A0vQa8WzivhA1o=,5,male,9,2006-09-13
1,S0x/syLRR27zhm5XkwLyOhKQZ5ZuAlPXLsvn+2zdKxo=,5,na,3,2014-07-11
2,c5uGl34Ab67ZhsRrU1gp7eH8mu3reHiWCoFSFe1GojI=,1,na,7,2013-01-23
3,5O3N5NsNwxiNFjs0CoIWq2nYXq1+qzb2xnebJx7abEA=,1,na,7,2011-02-06
4,N+/46c7mwmuHkCGg1RQAmzL1RD66isaYBMmJglPgy7I=,1,na,7,2016-01-16


### Transaction ~ Partitioned_Silver to Actual Silver (processed)

In [17]:
actual_silver_transaction_directory = "datamart/silver/transaction/"

In [18]:
for date_str in dates_str_lst:
    utils.silver.process_silver_table_transaction(date_str, bucket_name, bronze_transaction_directory, actual_silver_transaction_directory, spark)

loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_01_01.csv row count: 16177


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_01_01.parquet


loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_02_01.csv row count: 14710


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_02_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_03_01.csv row count: 18392


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_03_01.parquet


...

saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_11_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2016_12_01.csv row count: 22969


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_12_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_01_01.csv row count: 26778


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_01_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_02_01.csv row count: 19328


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_02_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_03_01.csv row count: 1823


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_03_01.parquet


### User Log ~ Bronze User Log to Silver User Log

In [19]:
silver_userlog_directory = "datamart/silver/userlog/"


# bronze_userlog_directory = "datamart/bronze/userlog"
# actual_silver_transaction_directory = "datamart/silver/transaction/"

In [20]:

for date_str in dates_str_lst:
    utils.silver.process_silver_table_userlog(date_str, bucket_name, bronze_userlog_directory, actual_silver_transaction_directory, silver_userlog_directory, spark)

loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_01_01.csv row count: 337141
Also loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_02_01.csv row count: 301479


saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_01_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_02_01.csv row count: 301479


Also loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_03_01.csv row count: 348811


saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_02_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_03_01.csv row count: 348811


Also loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_04_01.csv row count: 345465


...

loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_02_01.csv row count: 377934


Also loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv row count: 432095


saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_02_01.parquet


loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv row count: 432095
File gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_04_01.csv does not exist. Proceeding with single snapshot.


saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_03_01.parquet


# Gold ~ Feature Store

In [21]:
gold_feature_store_directory = "datamart/gold/feature_store/"

silver_userlog_directory = "datamart/silver/userlog/"
actual_silver_transaction_directory = "datamart/silver/transaction/"
silver_member_directory = "datamart/silver/member/"

In [22]:
for date_str in dates_str_lst:
    utils.gold.process_gold_featurestore(date_str, bucket_name, actual_silver_transaction_directory, silver_userlog_directory, silver_member_directory, gold_feature_store_directory, spark)

loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_01_01.parquet row count: 16177
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_01_01.parquet row count: 15059
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 50000
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+-------------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|membership_start_date|     discount_ratio|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+-------------------+
|++IU7zZpfAGO1qdHB...|               41|               30|          149.0|             119.0|

25/06/18 07:44:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2015_01_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_02_01.parquet row count: 14710
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_02_01.parquet row count: 13810
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 50000
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+-------------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|membership_start_date|     discount_ratio|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+-------------------+
|+

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2015_02_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_03_01.parquet row count: 18392
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_03_01.parquet row count: 17194
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 50000
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+-------------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|membership_start_date|     discount_ratio|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+-------------------+
|+

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2015_03_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_04_01.parquet row count: 15061
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_04_01.parquet row count: 13808
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 50000
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|membership_start_date|discount_ratio|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|++5nB0VuUuyj9xln

...

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2016_11_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_12_01.parquet row count: 22969
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2016_12_01.parquet row count: 21491
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 50000
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|membership_start_date|discount_ratio|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|++5wYjoMgQHoRuD3

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2016_12_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_01_01.parquet row count: 26778
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_01_01.parquet row count: 24947
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 50000
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|membership_start_date|discount_ratio|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|++5wYjoMgQHoRuD3

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2017_01_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_02_01.parquet row count: 19328
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_02_01.parquet row count: 18106
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 50000
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|membership_start_date|discount_ratio|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|++5wYjoMgQHoRuD3

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2017_02_01.parquet
loaded from: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_03_01.parquet row count: 1823
loaded from: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_03_01.parquet row count: 1647
loaded from: gs://cs611_mle/datamart/silver/member//silver_member.parquet row count: 50000
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|membership_start_date|discount_ratio|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+---------------------+--------------+
|+3gBpPePZrN6rcFLh.

saved to: gs://cs611_mle/datamart/gold/feature_store//gold_featurestore_2017_03_01.parquet


In [23]:


# # =========== 修改成你的 GCS 路径 ============

# transaction_parquet_path = f"gs://{bucket_name}/{actual_silver_transaction_directory}/silver_transaction_2015_01_01.parquet"
# userlog_parquet_path = f"gs://{bucket_name}/{silver_userlog_directory}/silver_userlog_2015_01_01.parquet"
# member_parquet_path = f"gs://{bucket_name}/{silver_member_directory}/silver_member.parquet"


# # ======== 读取 Parquet 文件为 DataFrame ========
# df_transaction = spark.read.parquet(transaction_parquet_path)
# df_userlog = spark.read.parquet(userlog_parquet_path)
# df_member = spark.read.parquet(member_parquet_path)

# # ======== 展示前 20 行 ========
# print("=== Transaction Table Preview ===")
# # df_transaction.show(20, truncate=False)
# df_transaction.toPandas().head()


# print("=== Userlog Table Preview ===")
# # df_userlog.show(20, truncate=False)
# df_userlog.toPandas().head()

# print("=== Member Table Preview ===")
# # df_member.show(20, truncate=False)
# df_member.toPandas().head()

# # ======== 保存为 CSV 文件 ========
# # df_transaction.write.mode("overwrite").option("header", "true").csv(transaction_csv_path)
# # df_userlog.write.mode("overwrite").option("header", "true").csv(userlog_csv_path)
# # df_member.write.mode("overwrite").option("header", "true").csv(member_csv_path)

# # print("✅ Conversion complete. CSV files saved.")

In [25]:
# print("=== Userlog Table Preview ===")
# # df_userlog.show(20, truncate=False)
# df_userlog.toPandas().head(20)

In [26]:
# print("=== Transaction Table Preview ===")
# # df_transaction.show(20, truncate=False)
# df_transaction.toPandas().head(20)